In [1]:
import numpy as np
import random
import gensim
import pandas as pd

### Carregando dataset

In [2]:
text = np.load('../../Corpora/PetroNER-LinkedEntity/sentences.npy')
entities = np.load('../../Corpora/PetroNER-LinkedEntity/entities.npy')
classes = np.load('../../Corpora/PetroNER-LinkedEntity/classes.npy')
URI =  np.load('../../Corpora/PetroNER-LinkedEntity/URI.npy')
URIvec =  np.load('../../Corpora/PetroNER-LinkedEntity/URI_vectors.npy')

### **A separação entre treino e teste deverá ser refeita depois que a PUC entregar o PetroNER dividido entre trieno e teste**

Iremos separar o conjunto de treino, validação e teste baseado nas URI. Ou seja, as URI observadas na fase de treinamento do modelo não será usada para a validação e teste.

In [3]:
# Listando as URI referente às classes que aparecerão nos datasets de treino, validação e teste

# Lista de URI que não serão separadas entre treino, validação e teste 
# Incluímos manualmente as URI das principais classes
URI_not_split = ['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#basin',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#field',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#well',
                 'http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#rock',
                 None]

# Fração das URI que não serão separadas entre treino, validação e teste 
fra_URI_not_split = 0.6

for U in set(URI):   
    if random.random() < fra_URI_not_split:
        URI_not_split.append(U)

URItre = []
URIval = []
URItes = []

# Separando as URI para treino (70%), validação (15%) e teste (15%)
for U in set(URI):   
    if U in URI_not_split:
        pass
    else:
        aleat = random.random()
        if aleat < 0.70:
            URItre.append(U)
        else:
            if aleat < 0.85:
                URIval.append(U)
            else:
                URItes.append(U)


In [4]:
# Separando os dataset de treino, validação e teste

# Listas de treino
text_treino = []
entities_treino = []
classes_treino = []
URI_treino = []
URIvec_treino = []

# Listas de validação
text_valid = []
entities_valid = []
classes_valid = []
URI_valid = []
URIvec_valid = []

# Listas de teste
text_teste = []
entities_teste = []
classes_teste = []
URI_teste = []
URIvec_teste = []

for n in range(len(URI)):
    # Verificando se a URI pertence a lista 'URI_not_split' distribuir usando a mesma proporção entre treino, validação e teste.
    if URI[n] in URI_not_split:
        aleat = random.random()
        if aleat < 0.70:
            text_treino.append(text[n])
            entities_treino.append(entities[n])
            classes_treino.append(classes[n])
            URI_treino.append(URI[n])
            URIvec_treino.append(URIvec[n])
            
        else:
            if aleat < 0.85:
                text_valid.append(text[n])
                entities_valid.append(entities[n])
                classes_valid.append(classes[n])
                URI_valid.append(URI[n])
                URIvec_valid.append(URIvec[n])
                
            else:
                text_teste.append(text[n])
                entities_teste.append(entities[n])
                classes_teste.append(classes[n])
                URI_teste.append(URI[n])
                URIvec_teste.append(URIvec[n])
                
    # Verificando se URI está na lista de treino
    if URI[n] in URItre:
        text_treino.append(text[n])
        entities_treino.append(entities[n])
        classes_treino.append(classes[n])
        URI_treino.append(URI[n])
        URIvec_treino.append(URIvec[n])
        
    # Verificando se URI está na lista de validação
    if URI[n] in URIval:
        text_valid.append(text[n])
        entities_valid.append(entities[n])
        classes_valid.append(classes[n])
        URI_valid.append(URI[n])
        URIvec_valid.append(URIvec[n])
        
    # Verificando se URI está na lista de teste
    if URI[n] in URItes:
        text_teste.append(text[n])
        entities_teste.append(entities[n])
        classes_teste.append(classes[n])
        URI_teste.append(URI[n])
        URIvec_teste.append(URIvec[n])
        
# Listas de treino
text_treino = np.array(text_treino)
entities_treino = np.array(entities_treino)
classes_treino = np.array(classes_treino)
URI_treino = np.array(URI_treino)
URIvec_treino = np.array(URIvec_treino)

# Listas de validação
text_valid = np.array(text_valid)
entities_valid = np.array(entities_valid)
classes_valid = np.array(classes_valid)
URI_valid = np.array(URI_valid)
URIvec_valid = np.array(URIvec_valid)

# Listas de teste
text_teste = np.array(text_teste)
entities_teste = np.array(entities_teste)
classes_teste = np.array(classes_teste)
URI_teste = np.array(URI_teste)
URIvec_teste = np.array(URIvec_teste)

In [5]:
print ('Treino')
print('Texto: ', len(text_treino))
print('Entidades: ', len(entities_treino))
print('Classes: ', len(classes_treino))
print('URI: ', len(URI_treino))
print('URIvec: :', len(URIvec_treino))

print ('\n Validação')
print('Texto: ', len(text_valid))
print('Entidades: ', len(entities_valid))
print('Classes: ', len(classes_valid))
print('URI: ', len(URI_valid))
print('URIvec: :', len(URIvec_valid))

print ('\n Teste')
print('Texto: ', len(text_teste))
print('Entidades: ', len(entities_teste))
print('Classes: ', len(classes_teste))
print('URI: ', len(URI_teste))
print('URIvec: :', len(URIvec_teste))

Treino
Texto:  7254
Entidades:  7254
Classes:  7254
URI:  7254
URIvec: : 7254

 Validação
Texto:  1199
Entidades:  1199
Classes:  1199
URI:  1199
URIvec: : 1199

 Teste
Texto:  1391
Entidades:  1391
Classes:  1391
URI:  1391
URIvec: : 1391


### Preparando dataset para treinar rede Siamesa

In [7]:
#Dataframe com as URI, sentenças e classes

def dataset_siamesa(classes, URI, URIvec, text, x_easy, x_hard):
       
    df = pd.DataFrame({'Classe': classes, 'URI': URI, 'Sentença': text}) #, 'URIvec': URIvec})
    
    # Arrays para armazenar as sentenças âncoras, positivas e negativas 
    URI_anchor = np.array([])
    URIvec_anchor = np.array([])
    dataset_anchor = np.array([])
    dataset_positive = np.array([])
    dataset_negative = np.array([])

    #Quantidade de exemplos negativos fáceis e difíceis
    #x_easy = 20
    #x_hard = 30

    # iterando por todas as URI
    for i in set(URI):

        try:
            
            # Subgrupo apenas com a URI selecionada
            df_URI = df[df['URI'] == i]

            # Classe da URI selecionada
            classe_da_URI = df_URI['Classe'].values[0]
            # Vetor da URI selecionada
            vetor_da_URI = URIvec[df_URI.index[0]]
            # Subgrupo com sentenças negativa 'fácil' (URI diferente e classe diferente).
            df_negative_easy = df[(df['URI'] != i) & (df['Classe'] != classe_da_URI)]
            # Subgrupo com sentenças negativa 'difícil' (URI diferente mas da mesma classe).
            df_negative_hard = df[(df['URI'] != i) & (df['Classe'] == classe_da_URI)]

            # Pular os casos onde só existe uma sentença para a URI selecionada
            if (len(df_URI) > 1) & (len(df_negative_easy) > 1) & (len(df_negative_hard) > 1):

                ### Construindo triplets com exemplos negativos fáceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_easy for maior que esse valor, amostramos apenas essa quantidade de amostras.
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1) , x_easy), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():

                    anchor = row[1]['Sentença']
                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = np.str_(df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values)

                    # Seleciona uma sentença negativa
                    negative_easy = np.str_(df_negative_easy.sample(1)['Sentença'].values)

                    # Concatena com as arrays que armazenam as senteças e URI
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=0)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative = np.concatenate((dataset_negative, np.array([negative_easy])), axis=0)

                ### Construindo triplets com exemplos negativos difíceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_hard for maior que esse valor, amostramos apenas essa quantidade de amostras.
                
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1), x_hard), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():
                    anchor = row[1]['Sentença']

                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = np.str_(df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values)

                    # Seleciona uma sentença negativa
                    negative_hard = np.str_(df_negative_hard.sample(1)['Sentença'].values)

                    # Concatena com as arrays que armazenam as senteças
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=1)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor  = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive  = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative  = np.concatenate((dataset_negative, np.array([negative_hard])), axis=0)

        except:
            pass
    
    #Ajustando o formato da array URIvec_anchor
    URIvec_anchor = np.reshape(URIvec_anchor, (-1, len(URIvec[0])))
    
    return(URI_anchor, URIvec_anchor, dataset_anchor, dataset_positive, dataset_negative)

In [ ]:
URI_anchor_treino, URIvec_anchor_treino, dataset_anchor_treino, dataset_positive_treino, dataset_negative_treino = dataset_siamesa(classes_treino,
                                                                                                                                   URI_treino,
                                                                                                                                   URIvec_treino,
                                                                                                                                   text_treino, 50, 50)
URI_anchor_valid, URIvec_anchor_valid, dataset_anchor_valid, dataset_positive_valid, dataset_negative_valid = dataset_siamesa(classes_valid,
                                                                                                                              URI_valid,
                                                                                                                              URIvec_valid,
                                                                                                                              text_valid, 50, 50)
URI_anchor_teste, URIvec_anchor_teste, dataset_anchor_teste, dataset_positive_teste, dataset_negative_teste = dataset_siamesa(classes_teste,
                                                                                                                              URI_teste,
                                                                                                                              URIvec_teste,
                                                                                                                              text_teste, 50, 50)

In [ ]:
print('Dataset treino: ', len(URI_anchor_treino))
print('Dataset validação: ', len(dataset_anchor_valid))
print('Dataset teste: ', len(dataset_anchor_teste))

### Salvando dataset


In [ ]:
#Treino
with open('URI_anchor_treino.npy', 'wb') as f:
    np.save(f, URI_anchor_treino)
with open('URIvec_anchor_treino.npy', 'wb') as f:
    np.save(f, URIvec_anchor_treino)
with open('anchor_treino.npy', 'wb') as f:
    np.save(f, dataset_anchor_treino)
with open('positive_treino.npy', 'wb') as f:
    np.save(f, dataset_positive_treino)
with open('negative_treino.npy', 'wb') as f:
    np.save(f, dataset_negative_treino)

    
#Validação
with open('URI_anchor_valid.npy', 'wb') as f:
    np.save(f, URI_anchor_valid)
with open('URIvec_anchor_valid.npy', 'wb') as f:
    np.save(f, URIvec_anchor_valid)
with open('anchor_valid.npy', 'wb') as f:
    np.save(f, dataset_anchor_valid)
with open('positive_valid.npy', 'wb') as f:
    np.save(f, dataset_positive_valid)
with open('negative_valid.npy', 'wb') as f:
    np.save(f, dataset_negative_valid)
    
#Teste
with open('URI_anchor_teste.npy', 'wb') as f:
    np.save(f, URI_anchor_teste)
with open('URIvec_anchor_teste.npy', 'wb') as f:
    np.save(f, URIvec_anchor_teste)
with open('anchor_teste.npy', 'wb') as f:
    np.save(f, dataset_anchor_teste)
with open('positive_teste.npy', 'wb') as f:
    np.save(f, dataset_positive_teste)
with open('negative_teste.npy', 'wb') as f:
    np.save(f, dataset_negative_teste)